In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter

KeyboardInterrupt: 

## Data Processing

In [2]:
df = pd.read_csv("job_descriptions.csv")

In [3]:
feature_to_filter_type = "Work Type"
values_to_keep_type = ["Full-Time"]

df_filtered_type = df[df[feature_to_filter_type].isin(values_to_keep_type)]


In [4]:
feature_to_filter_country = "Country"
values_to_keep_country = ["USA"]

df_filtered_country = df[df[feature_to_filter_country].isin(values_to_keep_country)]


In [5]:
import json

def extract_sector_industry(row):
  try:
    profile = json.loads(row['Company Profile'])
    return profile.get('Sector'), profile.get('Industry')
  except:
    return None, None

df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')


/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_80013/1242162936.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')
/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_80013/1242162936.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country[['Sector', 'Industry']] = df_filtered_country.apply(extract_sector_industry, axis=1, result_type='expand')


In [6]:
df_filtered_country.drop(columns=['Company Profile'], inplace=True)

/var/folders/d0/7dy3yx5n4v77cs4x81jpwwcc0000gn/T/ipykernel_80013/1005077129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_country.drop(columns=['Company Profile'], inplace=True)


In [7]:
df_filtered_country.drop(columns=['Job Id']).describe()

,latitude,longitude,Company Size
count,7.431000e+03,7.431000e+03,7431.000000
mean,3.709020e+01,-9.571290e+01,73073.289194
std,2.934739e-12,7.446989e-12,35201.315418
min,3.709020e+01,-9.571290e+01,12650.000000
25%,3.709020e+01,-9.571290e+01,42393.500000
50%,3.709020e+01,-9.571290e+01,72502.000000
75%,3.709020e+01,-9.571290e+01,103686.500000
max,3.709020e+01,-9.571290e+01,134824.000000


## Sector & Industry Prediction

In [8]:
import requests
import json

url = "http://localhost:11434/api/chat"

In [9]:
def llama3(prompt):
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()["message"]["content"]

In [10]:
sectors_list = ['Aerospace Building Systems', 'Aerospace and Defense', 'Agriculture', 
                'Airlines', 'Appliances and Electronics', 'Automotive', 
                'Automotive Motorcycles', 'Automotive Parts', 'Aviation and Travel', 
                'Banking', 'Beauty', 'Beverage Alcohol', 'Beverages', 'Business Services', 
                'Chemicals', 'Communication Equipment', 'Conglomerate', 
                'Construction Building Materials', 'Consumer Goods', 'Diversified', 
                'Electrical Equipment', 'Electronics', 'Energy', 'Energy Oil and Gas', 
                'Engineering', 'Entertainment', 'Equipment Rental', 'Financial Services', 
                'Financial Services Investment Management', 'Financial Technology', 
                'Financials', 'Flavor and Fragrances', 'Food and Beverage', 'Healthcare', 
                'Healthcare Services', 'Hospitality', 'Hospitality Hotels', 'IT Consulting', 
                'IT Services', 'Industrial Conglomerates', 'Industrial Machinery', 
                'Industrial Supplies', 'Information Technology', 'Infrastructure', 
                'Insurance', 'Lighting and Technology', 'Logistics', 'Lubricants', 
                'Manufacturing', 'Media', 'Media Entertainment', 'Medical Devices', 
                'Medical Technology', 'Metals', 'Metals and Mining', 'Mining and Metals Gold', 
                'Motion Control', 'Natural Gas', 'Oil and Gas', 'Oil and Gas Services', 
                'Packaging', 'Payroll   HR Services', 'Pharmaceuticals', 'Postal Services', 
                'Real Estate', 'Real Estate Construction', 'Restaurants', 'Retail', 
                'Retail and Conglomerate', 'Services', 'Shipping and Logistics', 
                'Software and Technology', 'Technology', 'Technology IT Services', 
                'Technology and Networking', 'Technology and Search', 
                'Technology and Software', 'Telecommunications', 'Timber   Paper', 'Tobacco', 
                'Transportation', 'Travel and Booking', 'Utilities']


In [11]:
print(sectors_list.__len__())

83


In [12]:
industry_list = ['Real Estate' 'Aerospace and Defense' 'Chemicals' 'Commercial Banks'
 'Utilities: Gas and Electric' 'Pharmaceuticals' 'Airlines' 'Beverages'
 'Petroleum Refining' 'Banking & Financial Services' 'Entertainment'
 'Internet Services and Retailing' 'Real Estate/REIT'
 'Information Technology' 'Semiconductors and Other Electronic Components'
 'Financial Data Services' 'Food Services' 'Automotive'
 'Electrical Equipment' 'Travel and Leisure - Hotels'
 'Energy - Oil & Gas Exploration & Production'
 'Mining, Crude-Oil Production' 'Financial Services'
 'Wholesalers: Diversified' 'Financial Services - Investment'
 'Transportation/Logistics' 'Wholesalers: Health Care' 'Mining'
 'Automotive Retailing, Services' 'Telecommunications'
 'Diversified Financials' 'Packaging, Containers' 'Industrial Machinery'
 'Business Services - Pest Control' 'Technology & Telecommunications'
 'Aerospace & Defense' 'Food and Drug Stores' 'Food Production'
 'Pipelines' 'Consumer Goods' 'Specialty Retailers: Other'
 'Home Equipment, Furnishings' 'Tobacco' 'Health Care: Medical Facilities'
 'Energy' 'Industrial Conglomerate' 'Electronics & Appliances'
 'General Merchandisers' 'Diversified Outsourcing Services' 'Oil and Gas'
 'Equipment Rental' 'Airlines & Aviation' 'Media and Entertainment'
 'Banking' 'Insurance: Life, Health (Stock)' 'Food & Beverage'
 'Forest and Paper Products' 'Banking/Financial Services'
 'Postal Services' 'Insurance - Property/Casualty'
 'Insurance: Property and Casualty (Stock)'
 'Food and Beverage - Alcoholic Beverages' 'Energy - Oil & Gas Services'
 'Metals' 'Food Consumer Products' 'Retail' 'Transportation and Logistics'
 'Medical Products and Equipment' 'Information Technology Services'
 'Computer Software' 'Engineering & Construction' 'Utilities'
 'Health Care: Insurance and Managed Care' 'Securities'
 'Paints and Coatings' 'Gas and Energy' 'Trucking, Truck Leasing'
 'Healthcare' 'Restaurants' 'Insurance' 'Technology' 'Internet/Technology'
 'Building Materials/Construction' 'Electronics/Components'
 'Financial Services/Insurance' 'Software' 'Lighting & Electronics'
 'Network and Other Communications Equipment'
 'Technology & Search Engines' 'Wholesalers: Food and Grocery'
 'Publishing, Printing' 'Household and Personal Products'
 'Engineering/Industrial Equipment' 'Food and Beverage'
 'Insurance: Life, Health (Mutual)' 'Motor Vehicles & Parts' 'Oil & Gas'
 'Health Care: Pharmacy and Other Services' 'Retail - Books & News'
 'Real Estate/Construction' 'Food Manufacturing'
 'Hotels, Casinos, Resorts' 'Retail - Farm and Ranch Supplies'
 'Diversified' 'Cement and Building Materials'
 'Logistics & Delivery Services' 'Banking and Financial Services'
 'Wholesalers: Electronics and Office Equipment'
 'Manufacturing - Diversified' 'Energy - Diversified'
 'Building Materials, Glass' 'Mail, Package and Freight Delivery'
 'Hospitality & Entertainment' 'Conglomerate' 'Gambling/Gaming'
 'Computers, Office Equipment' 'Travel and Leisure - Cruises'
 'Temporary Help' 'Financial Services/Investment Management'
 'Apparel and Textiles' 'Homebuilders' 'Steel'
 'Technology & Entertainment' 'Personal Care & Cosmetics' 'Apparel'
 'Manufacturing/Building Systems' 'Distribution/Wholesale'
 'Transportation - Rail' 'Technology & Social Media' 'Energy - Utilities'
 'Consumer Products - Cleaning Products' 'Advertising, Marketing'
 'Waste Management' 'E-commerce & Technology' 'Construction/Engineering'
 'Railroads' 'Financial Services - Investment Banking'
 'Retail - Food & Drug' 'Specialty Retailers: Apparel'
 'Industrial Manufacturing' 'Cement and Concrete' 'Tire Manufacturing'
 'Insurance: Property and Casualty (Mutual)' 'Beverage'
 'Construction and Farm Machinery'
 'Scientific, Photographic and Control Equipment'
 'Advertising and Marketing' 'Manufacturing - Electrical Equipment'
 'Financial Technology (Fintech)' 'Transportation - Airlines'
 'Information Technology and Services' 'Electronics, Electrical Equip.'
 'Transportation/Infrastructure' 'Electronics/Instrumentation'
 'Apparel/Fashion' 'Agrochemicals' 'Hospitality/Hotels'
 'E-commerce/Technology' 'Retail - Discount/Department Stores' 'Logistics'
 'Manufacturing & Transportation' 'Retail - General Merchandise'
 'Retail - Home Improvement' 'Transportation' 'Medical Devices/Healthcare'
 'Transportation Equipment' 'Ports and Infrastructure'
 'Manufacturing/Steel' 'Entertainment - Streaming Services'
 'Business Services - Testing/Compliance' 'Semiconductors'
 'Technology & Electronics' 'Consumer Products - Cosmetics'
 'Online Recruitment' 'Entertainment - Satellite Radio' 'Retail - Apparel'
 'Construction' 'Retail/Home Improvement' 'Jewelry and Watches'
 'Renewable Energy' 'Gaming/Gambling' 'Software/Technology'
 'Oil and Gas Equipment, Services' 'Gambling'
 'Business Services - Data/Analytics' 'Energy - Coal' 'Aviation/Airlines'
 'Retail - Electronics' 'Technology - Ride-hailing'
 'Construction & Building Materials' 'Education' 'Life Sciences'
 'Cement and Chemicals' 'Entertainment - Music Streaming' 'E-commerce'
 'Construction and Engineering' 'Media & Entertainment' 'Electronics'
 'Equipment Leasing' 'Technology & Security'
 'Aluminum and Copper Manufacturing' 'Apparel & Footwear'
 'Cloud Software and Services' 'Media/Entertainment']


In [13]:
print(industry_list.__len__())

1


In [14]:
def getSector(prompt):
    sectors_str = ", ".join(sectors_list)
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                # What we are asking the API, will be given the name of company in the prompt
                "content": f"Identify the sector for the company named {prompt}. " 
                            "Respond strictly in the format: 'Sector: [best matching sector]'. "
                            f"Select the most appropriate sector from this list: {sectors_str}." 
                            "Do not include any additional information or commentary."
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    response = response.json()["message"]["content"]
    
    # extract sector and industry
    try:
        sector = str(response).split("Sector: ")[1].split(",")[0]
    except IndexError:
        sector = "Unknown"

    return sector, 


In [15]:
def getIndustry(prompt):
    industry_str = ", ".join(industry_list)
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                # What we are asking the API, will be given the name of company in the prompt
                "content": f"Identify the industry for the company named {prompt}. " 
                            "Respond strictly in the format: 'Industry: [best matching industry]'. "
                            f"Select the most appropriate industry from this list: {industry_str}." 
                            "Do not include any additional information or commentary."
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    response = response.json()["message"]["content"]
    
    # extract sector and industry
    try:
        industry = str(response).split("Industry: ")[1].split("]")[0].strip()
    except IndexError:
        industry = "Unknown",

    return industry


In [16]:
response = llama3("who wrote the book godfather")
print(response)

The book "The Godfather" was written by Mario Puzo. The novel was published in 1969 and became a huge success, telling the story of the Corleone crime family's rise to power and their struggles with rival families and the law.

Mario Puzo was an American author known for his novels about the Italian-American Mafia. He is best known for writing "The Godfather" novel, which was later adapted into the iconic film directed by Francis Ford Coppola.

Puzo wrote several other novels based on the Corleone family, including "The Last Don," "The Fourth K," and "The Sicilian." He also collaborated with Coppola to write the screenplays for the first two Godfather films. Puzo passed away in 1999 at the age of 78.

It's worth noting that while Puzo wrote the original novel, the film adaptation of "The Godfather" was a collaborative effort between Puzo and Coppola. The screenplay for the film was written by Coppola, Puzo, and others, and it is widely regarded as one of the greatest films of all time.

In [17]:
response = getSector("Apple")
reponse_industry = getIndustry("Apple")
print(response)
print(reponse_industry)

('Appliances and Electronics',)
Information Technology


In [18]:
companies = df_filtered_country['Company']

In [20]:
predicted_sectors = []
predicted_industries = []
for company in companies[:50]:
    predicted_sector = getSector(company)
    predicted_industry = getIndustry(company)
    predicted_sectors.append(predicted_sector)
    predicted_industries.append(predicted_industry)

In [31]:
print(predicted_sectors)
predicted_sectors = ['Real Estate', 'Aerospace and Defense', 'Flavor and Fragrances', 'Financials', 'Energy', 'Pharmaceuticals', 'Airlines', 'Beverage Alcohol', 'Energy Oil and Gas', 'Banking', 'Entertainment', 'Travel and Booking', 'Real Estate', 'Information Technology', 'Electronics', 'Financial Technology', 'Consumer Goods', 'Airlines', 'Automotive', 'Chemicals', 'Electrical Equipment', 'Hospitality', 'Energy Oil and Gas', 'Energy', 'Financial Services Investment Management', 'Energy Oil and Gas', 'Financials', 'Energy', 'Financials', 'Shipping and Logistics', 'Healthcare', 'Automotive', 'Metals and Mining', 'Automotive', 'Telecommunications', 'Financial Services', 'Packaging', 'Conglomerate', 'Energy', 'Business Services', 'Communication Equipment', 'Aerospace and Defense', 'Retail', 'Agriculture', 'Telecommunications', 'Energy', 'Energy Oil and Gas', 'Consumer Goods', 'Energy', 'Retail']
print(predicted_sectors)

['Real Estate', 'Aerospace and Defense', 'Flavor and Fragrances', 'Financials', 'Energy', 'Pharmaceuticals', 'Airlines', 'Beverage Alcohol', 'Energy Oil and Gas', 'Banking', 'Entertainment', 'Travel and Booking', 'Real Estate', 'Information Technology', 'Electronics', 'Financial Technology', 'Consumer Goods', 'Airlines', 'Automotive', 'Chemicals', 'Electrical Equipment', 'Hospitality', 'Energy Oil and Gas', 'Energy', 'Financial Services Investment Management', 'Energy Oil and Gas', 'Financials', 'Energy', 'Financials', 'Shipping and Logistics', 'Healthcare', 'Automotive', 'Metals and Mining', 'Automotive', 'Telecommunications', 'Financial Services', 'Packaging', 'Conglomerate', 'Energy', 'Business Services', 'Communication Equipment', 'Aerospace and Defense', 'Retail', 'Agriculture', 'Telecommunications', 'Energy', 'Energy Oil and Gas', 'Consumer Goods', 'Energy', 'Retail']
['Real Estate', 'Aerospace and Defense', 'Flavor and Fragrances', 'Financials', 'Energy', 'Pharmaceuticals', 'Air

In [32]:
df_comparison = pd.DataFrame({
    'company': df_filtered_country['Company'][:50],
    'predicted_sector': predicted_sectors,
    'sector': df_filtered_country['Sector'][:50],
    'predicted_industry': predicted_industries,
    'industry': df_filtered_country['Industry'][:50]
})

In [33]:
df_comparison['sector_correct'] = df_comparison['predicted_sector'] == df_comparison['sector']
df_comparison['industry_correct'] = df_comparison['predicted_industry'] == df_comparison['industry']


In [34]:
sector_accuracy = df_comparison['sector_correct'].mean()
industry_accuracy = df_comparison['industry_correct'].mean()

In [35]:
print(f"Accuracy for sector: {sector_accuracy}")
print(f"Accuracy for industry: {industry_accuracy}")

Accuracy for sector: 0.5
Accuracy for industry: 0.44


In [36]:
df_comparison

,company,predicted_sector,sector,predicted_industry,industry,sector_correct,industry_correct
156,Lendlease Group,Real Estate,Real Estate/Construction,Real Estate,Real Estate,False,True
270,United Technologies Corporation,Aerospace and Defense,Aerospace/Building Systems,Aerospace and Defense,Aerospace and Defense,False,True
1140,International Flavors & Fragrances,Flavor and Fragrances,Flavors & Fragrances,Chemicals,Chemicals,False,True
1591,KeyCorp,Financials,Financial Services,Commercial Banks,Commercial Banks,False,True
1609,Exelon,Energy,Energy,Utilities: Gas and Electric,Utilities: Gas and Electric,True,True
1668,Zoetis,Pharmaceuticals,Healthcare,Pharmaceuticals,Pharmaceuticals,False,True
1860,Southwest Airlines,Airlines,Airlines,Airlines,Airlines,True,True
1923,Diageo,Beverage Alcohol,Beverages,Beverages,Beverages,False,True
2236,PBF Energy,Energy Oil and Gas,Energy,Petroleum Refining,Petroleum Refining,False,True
2278,China Merchants Bank,Banking,Banking,Commercial Banks,Banking & Financial Services,True,False


We found that the accuracies for sectors and industires are low, however, we think that our model is still pretty good, considering that there are so many industries and sectors to choose from. We think that by narrowing down the pool of sectors and industries we are choosing from. Additioantly for example our model predicted a sector to be aviation when the real sector was airlines, these two are essentially the same, we think that this issue can be solved by creating a list of a few more general sectors and industuies and mapping each of the current sectors. 

After splitting into two function we were able to get a much better accuracy, 50% for sector and 44% for industry. This is a much better result from before, we are still thinking of condensing the list of sectors and industry to a few general ones to increase accuracies.